In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import onnxruntime as ort
import os
import onnx
from PIL import Image
import cv2
from tqdm import tqdm
import struct
import matplotlib.pyplot as plt
import matplotlib as mpl
import json

mpl.rcParams['figure.figsize'] = (18, 18)
mpl.rcParams['axes.grid'] = False

import onnxruntime as ort

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

onnx_model_path = 'model_handdraw_bg.onnx'
onnx_model = onnx.load(onnx_model_path)
ort_session = ort.InferenceSession(onnx_model_path,providers=['CUDAExecutionProvider'])


In [ ]:
srcimg = cv2.imread('input.png').astype(np.float32)

outputs = ort_session.run(None, {'input_image:0': srcimg})
plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.imshow( cv2.imread('input.png')[:,:,::-1])
plt.subplot(1,2,2)
plt.imshow(outputs[0].astype(np.uint8)[:,:,::-1])


In [ ]:
# make batch convert scripts
AVAILABLE_GPU=0

out_scripts_fn = './batch_cvt_to_onnx.sh'
out_scripts = open(out_scripts_fn, 'w')
src_dir = './damo'
dst_dir = './onnx_models'
for model_tag in [
    '3d',
    'artstyle',
    'handdrawn',
    'sd-design',
    'sd-illustration',
    'sketch']:
    src_model_subdir = os.path.join(src_dir,f'cv_unet_person-image-cartoon-{model_tag}_compound-models')
    dst_model_subdir = os.path.join(dst_dir,f'cartoon-{model_tag}')
    # bg
    src_bg_model = os.path.join(src_model_subdir,'cartoon_bg.pb')
    dst_bg_model = os.path.join(dst_model_subdir,f'{model_tag}_bg.onnx')
    # head 
    src_head_model = os.path.join(src_model_subdir,'cartoon_h.pb')
    dst_head_model = os.path.join(dst_model_subdir,f'{model_tag}_h.onnx')
    os.makedirs(os.path.dirname(dst_bg_model),exist_ok=True)
    input_name = "input_image:0"
    output_name = "output_image:0"
    cvt_line =f'CUDA_VISIBLE_DEVICES={AVAILABLE_GPU} python -m tf2onnx.convert --graphdef {src_bg_model} --output {dst_bg_model} --inputs {input_name} --outputs {output_name}\n'
    out_scripts.write(cvt_line)
    cvt_line =f'CUDA_VISIBLE_DEVICES={AVAILABLE_GPU} python -m tf2onnx.convert --graphdef {src_head_model} --output {dst_head_model} --inputs {input_name} --outputs {output_name}\n'
    out_scripts.write(cvt_line)


out_scripts.close()